In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
from census import Census
# Import API key
from dotenv import load_dotenv
import os
#adds variable from environments - may need to pip install python-dotenv
load_dotenv()

# Establish Key Variables
api_key = os.environ.get("api_key")
if api_key == "None":
    api_key = input("Please enter a valid US Census API Key: ")

### Pull Census API Data

In [2]:
## This kernel pulls census data directly from the US Census API - the output is then used to transform the data below. If no US CENSUS API key available, please skip this Kernel and use previous CSV file for SQL output.
try:
    # Run Census Search to retrieve data on all states
    State_Census_Data = Census(api_key, year=2019).acs1.get(("NAME",
    "B19013_001E",
    "B01003_001E",
    "B01002_001E",
    "B19301_001E",
    "B17001_002E",
    "B23025_005E",
    "B27020_006E",
    "B27020_012E",
    "B27020_017E"),
    {'for': 'state:*'})

    # Convert to DataFrame
    State_Census_df = pd.DataFrame(State_Census_Data)
    # Calculates the total of uninsured residents into a new column
    State_Census_df["No_Ins"] = State_Census_df["B27020_006E"] + State_Census_df["B27020_012E"] + State_Census_df["B27020_017E"]

    # Column Reordering
    State_Census_df = State_Census_df.rename(columns={"B01003_001E": "Population",
    "B01002_001E": "Median Age",
    "B19013_001E": "Household Income",
    "B19301_001E": "Per Capita Income",
    "B17001_002E": "Poverty Count",
    "B23025_005E": "Unemployment Count",
    "B27020_006E": "Native Born with no health insurance coverage",
    "B27020_012E": "Foreign Born with no health insurance coverage",
    "B27020_017E": "Noncitizen with no health insurance coverage",
    "No_Ins": "Uninsured Residents",
    "NAME": "State", "state": "State ID"})

    # Add in Poverty Rate (Poverty Count / Population)
    State_Census_df["Poverty Rate"] = 100 * \
        State_Census_df["Poverty Count"].astype(
            int) / State_Census_df["Population"].astype(int)

    # Add in Employment Rate (Employment Count / Population)
    State_Census_df["Unemployment Rate"] = 100 * \
        State_Census_df["Unemployment Count"].astype(
            int) / State_Census_df["Population"].astype(int)

    # Final Output DataFrame
    State_Census_df = State_Census_df[["State", 
    "Population",
    "Median Age",
    "Household Income",
    "Per Capita Income",
    "Poverty Count",
    "Poverty Rate",
    "Unemployment Rate", 
    "Uninsured Residents"]]

    # Sort Dataframe by State and reset index
    State_Census_df = State_Census_df.sort_values(by="State")
    State_Census_df.reset_index(drop=True, inplace=True)

    # Outputs datframe to CSV file
    State_Census_df.to_csv("data/state_census_data.csv", encoding="utf-8", index=False)
    State_Census_df.head()
except:
    print("There was an error with the API Data Import - will continue using previously outputted CSV Data")

### Extract CSVs into DataFrames

In [3]:
# Extract CSVs into DataFrames
State_Census_URL = "data/state_census_data.csv"
State_Census_df = pd.read_csv(State_Census_URL)
State_Census_df.count()

State                  52
Population             52
Median Age             52
Household Income       52
Per Capita Income      52
Poverty Count          52
Poverty Rate           52
Unemployment Rate      52
Uninsured Residents    52
dtype: int64

In [4]:
# Extract CSVs into DataFrames
covid_file = "data/inpatient_covid.csv"
covid_df = pd.read_csv(covid_file)
covid_df.count()

state                                                                   1696
collection_date                                                         1696
Inpatient Beds Occupied by COVID-19 Patients Estimated                  1696
Count LL                                                                1696
Count UL                                                                1696
Percentage of Inpatient Beds Occupied by COVID-19 Patients Estimated    1696
Percentage LL                                                           1696
Percentage UL                                                           1696
Total Inpatient Beds                                                    1696
Total LL                                                                1696
Total UL                                                                1696
dtype: int64

In [5]:
# Extract CSVs into DataFrames
death_data = "data/covid_deaths_by_state.csv"
death_df = pd.read_csv(death_data)
death_df.count()

submission_date    14640
state              14640
tot_cases          14640
conf_cases          4697
prob_cases          4697
new_case           14640
pnew_case           9376
tot_death          14640
conf_death          5008
prob_death          5008
new_death          14640
pnew_death          9374
created_at         14640
consent_cases      11712
consent_deaths     11956
dtype: int64

## Transform Data

In [6]:
# Transform DataSet 1
# Abbreviation Dictionary
abbrev_dict = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}
# Remove DC and Puerto Rico to include only 50 states
State_Census_df = State_Census_df[State_Census_df["State"] != "District of Columbia"]
State_Census_df = State_Census_df[State_Census_df["State"] != "Puerto Rico"]
# Change State names to abbreviations
State_Census_df = State_Census_df
State_Census_df["State"] = State_Census_df["State"].map(abbrev_dict)
State_Census_df.head()

,State,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,Uninsured Residents
0,AL,4903185.0,39.4,51734.0,28650.0,739108.0,15.074039,2.276520,468901.0
1,AK,731545.0,35.0,75463.0,36978.0,71938.0,9.833708,2.822793,86187.0
2,AZ,7278717.0,38.3,62055.0,32173.0,958442.0,13.167733,2.429934,808643.0
3,AR,3017804.0,38.8,48952.0,27274.0,474739.0,15.731273,2.224863,270970.0
4,CA,39512223.0,37.0,80440.0,39393.0,4552837.0,11.522604,2.575557,3001997.0


In [7]:
# Transform DataSet 2
drop_col = covid_df.drop(columns={'Count LL', 'Count UL','Percentage LL', 'Percentage UL', 'Total LL', 'Total UL'})
drop_col

# Filter DataFrame from specific columns
rename_cols = drop_col.rename(columns={"Inpatient Beds Occupied by COVID-19 Patients Estimated":"INPT Bed Estimate",
                                      "Percentage of Inpatient Beds Occupied by COVID-19 Patients Estimated":"PCT INPT Bed Estimate"})
fifty_states = rename_cols[(rename_cols.state != 'CW' ) & (rename_cols.state != 'PR') & (rename_cols.state != 'DC')]

Covid_Beds = fifty_states.groupby(['state'], sort=True).agg({"collection_date":"max"}).reset_index()

Covid_Beds.merge(fifty_states, on=['state', 'collection_date'])

Covid_Beds.head()

,state,collection_date
0,AK,2020-09-20
1,AL,2020-09-20
2,AR,2020-09-20
3,AZ,2020-09-20
4,CA,2020-09-20


In [8]:
# Transform DataSet 3
#revoming unneccessary columns  
updated_df = death_df.drop(columns = ['created_at', 'prob_cases', 'pnew_case', 'prob_death', 'pnew_death', 
                                      'consent_cases', 'consent_deaths', 'new_death', 'conf_death', 'conf_cases'])

#renaming columns to proper names 
renamed_df = updated_df.rename(columns={'submission_date': 'Submission Date', 'state': 'State', 
                                       'tot_cases': 'Total Cases', 'conf_cases' : 'Confirmed Cases',
                                       'new_case' : 'New Cases', 'tot_death' : 'Total Deaths', 
                                        'conf_death' : 'Confirmed Deaths'})

#dropping 0 case submissions before the first reported date 
clean_df = renamed_df.dropna(thresh=5)

#look through dataframe and only keep values where total cases =! 0
#if total cases = 0, drop row 
updated_df = clean_df[clean_df['Total Cases'] != 0]

#look through data frame to ensure that all states are of the 50 states 
#if state does not equal actual state abbreviation , drop row 
updated_df['State'].unique()

#create list of states 
state_list = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL'
,'GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN'
,'MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR'
,'PA','RI','SC','SD','TN','TX','UT' ,'VT','VA','WA','WV', 'WI', 'WY']

#drop territories and other abbreviations not in the 50 states list 
Covid_Deaths = updated_df[updated_df['State'].isin(state_list)]

Covid_Deaths['State'].unique()

#display final df 
Covid_Deaths.head()

,Submission Date,State,Total Cases,New Cases,Total Deaths
43,03/05/2020,CO,1,1,0
44,03/06/2020,CO,2,1,0
45,03/07/2020,CO,8,6,0
46,03/08/2020,CO,8,0,0
47,03/09/2020,CO,12,4,0


### Create database connection

In [12]:
# Connects to SQL database to write tables
db_url = "postgres:postgres@localhost:5432/covid_census_db"
engine = create_engine(f'postgresql://{db_url}')

### Load DataFrames into database

In [13]:
# Assigns dataframe to table in SQL database
State_Census_df.to_sql(name='state_census', con=engine, if_exists='append', index=True)

In [14]:
# Assigns dataframe to table in SQL database
Covid_Beds.to_sql(name='covid_beds', con=engine, if_exists='append', index=True)

In [15]:
# Assigns dataframe to table in SQL database
Covid_Deaths.to_sql(name='covid_deaths', con=engine, if_exists='append', index=True)